## konlpy로 synop 명사 추출
## CountVectorizer, cosine_similarity

- 시놉 내 단어에 의한 유사도 계산
- 1차 불용어 제거 : tokenizer 2차 불용어 제거 min_df 값 조정
    ('min_df = x' # 단어장에 포함되기 위한 최소 빈도 x)
- csv의 id값이 1부터 시작하기 때문에 새 컬럼에 입력 시ipynb 결과의 index +1 해야함. 

+++ 추가 작업
- 다른 방송국 전체 데이터 모델 적용
- TfidfVectorizer은 다른 nb에 작성

In [1]:
import pandas as pd
import numpy as np
import unicodedata
import nltk
from nltk.corpus import stopwords
from konlpy.tag import *
#from ast import literal_eval

In [2]:
metadata = pd.read_csv('new_all_data_ver3.csv', encoding='cp949')
del metadata['synop']

In [3]:
metadata.head(1)

,idx,tv,title,epi,dates,links,vod_key,n_synop
0,1,SBS,SBS 스페셜,그랜드 슬램 대탐험 걸어서 지구 끝까지,2005.07.10,http://programs.sbs.co.kr/culture/sbsspecial/v...,"움직', '프로그램', '블리자드', '순간', '그랜드슬램'",스페셜 1편 그랜드 슬램 대탐험걸어서 지구 끝까지기획: 장광호김해영 촬영 (탐험대원...


In [4]:
metadata['epi'][0]  #csv 파일의 id = 1

'그랜드 슬램 대탐험  걸어서 지구 끝까지'

In [5]:
metadata['n_synop'][1] #csv 파일의 id = 2

'국민 열광 두 젊은이 그라운드 승부사인 박주영 박지성 수 두 젊은이 번 난관 끝 포기 노력 연습 실력 공통점 공 마음 일 거 당연 일 박주영 신드롬 명승부 명장면 월드컵 예선 청소년축구대회 그 명승부 골 국민들 환호 이후 박주영 소재 패러디 인터넷 등장 박주영 응원가 것 그 리그 서울 관중수 예전 약 두 배 이 박주영 반응 마디 쿨하 주위 반응 아랑 곳하지 자기 일 전형적 신세대 놀이축구 성공 원 그 축구 운동장 엄마 시장 친구 때 축구공 환상적 그 드리볼 때 축구화 맨발 축구 덕분 실력 목표 달성 축구 것 세상 일 축구 축구만 생각 박주영 이전 선수들 코드 축구스타 그 정신적 지주 두 여자 박주영에게 힘 두 여자 사람 아들 기도 어머니 사람 굼벵이 별명 그 여자친구 전반 시즌 마지막 경기 열리 7월 10일 이날 박주영 스무번 생일 이날 관중석 선글래스 경기 관전 한 여인 수 박주영 어머니 것 자신 아들 그림자 존재 이날 박주영 여자친구 지갑 자랑 최근 자신 홈페이지 여자친구 사진 공개 그녀 따뜻한 애정 한편 두 사람 사연 이번 첫 공개 박주영 목표 박지성 박주영 꿈 잉글랜드 진출 것 최근 맨체스터 유나이티드팀 한 박지성 그 꿈 박지성 세계 인정 스타 고등학교때 눈 수 프로팀 대학팀 그 지도한 고등학교 감독 평소 분 명지대 감독 박지성 부탁 정도 그 좌절 력 올림픽대표팀 허정무 감독 눈 리그 진출 2002 한일 월드컵 스타 벤치 그라운드 딩크 감독 감독 부임 네덜란드 아인트호벤팀 박지성 한번 위 홈팬들 아시아 실력 수 노골적 야유 그 부진 1년 반 히딩크 감독 비난 그 출장 박지성 세계적 스타 수 맨유팀 퍼거슨 감독 박지성 선택 것 그 녹초 이상 수 때 수 때문 의지 포기 태극전사 피 그 것 이 우려 벤치신세 대 그라운드 장악하 그 꿈 꿈 두 젊은이 꿈 것 매순간 노력 한국 축구 역사 두 젊은이 두 청년 꿈 순간 우리 그 인생 축구장 우리 좌절 꿈'

In [6]:
metadata['epi'][641] #csv 파일의 id = 642

'산에서 암을 이긴 사람들 2 - “자연식의 비밀”'

In [7]:
n_synoplist[641] #csv 파일의 id = 642

NameError: name 'n_synoplist' is not defined

In [8]:
print(len(metadata['n_synop']))
print(len(metadata['epi']))

1890
1890


In [9]:
# CountVectorizer하기 위해 n_synoplist 리스트 만듦.

n_synoplist=metadata['n_synop']

# CountVectorizer 

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

#10개 이하로 나오는 단어 제외
# count_matrix는 벡터행렬
count = CountVectorizer(min_df=10)
count_matrix = count.fit_transform(metadata['n_synop'].values.astype('U'))

In [11]:
count_matrix.todense()[0]

matrix([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## 코사인 유사도 점수 계산

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

#count_matrix에 n_synp을 리스트로 담았기 때문에 유사도 계산 시 count_matrix, count_matrix 형태가 됨)

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [13]:
# 코사인 유사도 값 출력
# 문장이 길고, 문장 내 단어가 다양해서 대체적으로 점수가 높지 않음.
cosine_sim

array([[1.        , 0.05421667, 0.1184329 , ..., 0.06018271, 0.        ,
        0.0446233 ],
       [0.05421667, 1.        , 0.16336601, ..., 0.08993385, 0.01683588,
        0.05878964],
       [0.1184329 , 0.16336601, 1.        , ..., 0.12490455, 0.01414496,
        0.04116098],
       ...,
       [0.06018271, 0.08993385, 0.12490455, ..., 1.        , 0.0700819 ,
        0.01812747],
       [0.        , 0.01683588, 0.01414496, ..., 0.0700819 , 1.        ,
        0.        ],
       [0.0446233 , 0.05878964, 0.04116098, ..., 0.01812747, 0.        ,
        1.        ]])

In [14]:
# 데이터 셋의 id는 1부터, dataframe의 index는 0부터
#index는 dataframe의 idx
indices = pd.Series(metadata.index, index=metadata['idx'])

In [15]:
def get_recommendations(idx, cosine_sim=cosine_sim):
    #제목과 일치하는 에피소드 인덱스 가지고옴
    idx = indices[idx]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #비슷한 에피소드 상위 5개
    sim_scores = sim_scores[1:11]
    
    # indices
    epi_indices = [i[0] for i in sim_scores]
    
        
    return metadata['epi'].iloc[epi_indices]

## Test get_recommendations(idx) : metadata['idx']를 따름

In [137]:
metadata['epi'][1]

'꿈을 쏘다! - 박주영 박지성'

In [16]:
get_recommendations(1) #컬럼의 인덱스와 idx값이 달라 +1 해줘야함. 실제 입력값 id = 2

380    북극열전-3부 원시북극의 위기(Entering Virgin Territory)
784                             북극열전 - 제4부 원시북극의 위
384              북극열전-6부 영토 분쟁의 서곡(Border Confict)
382             북극열전-4부 남획의 그늘(Predatory activies)
783                              북극열전 - 제3부 남획의 그늘
785                           북극열전 - 제5부 영토 분쟁의 서곡
378                             북극열전-1부 위기의 북극을 가다
379                      북극열전-2부 북극항로, 신해상 실크로드의 꿈
779                                북극열전 - 제1부 북극항로
383                북극열전-5부 신 냉전의 그림자(The War Game)
Name: epi, dtype: object

In [139]:
metadata['epi'][1391] #컬럼의 인덱스와 idx값이 달라 +1 해줘야함. 실제 입력값 id =1392

'밥상, 상식을 뒤집다- 탄수화물의 경고 1부'

In [17]:
get_recommendations(1391) #컬럼의 인덱스와 idx값이 달라 +1 해줘야함. 실제 입력값 id =1392

1255         병원의 고백1부 - 너무나 친절한 의사들
1258          병원의 고백 2부. 의사에게 길을 묻다
1441                  젊음도 성형할 수 있나요
1450                        간호사의 고백
1337    암(癌) 전문의의 선택 진료실에서 못다 한 이야기
261                      목숨 걸고 편식하다
1670                생존의 조건 - 권역외상센터
929           그녀, 뼈를 깎다 - 내 딸의 양악수술
940            내 몸 속에는 100명의 의사가 있다
1641                    건강할 권리를 찾아서
Name: epi, dtype: object

In [18]:
# 입력값인 에피소드 인덱스에 -1 해줘야 csv 데이터 셋 id와 일치.

print('episode : ' + metadata['epi'][1-1])
get_recommendations(1, cosine_sim)

episode : 그랜드 슬램 대탐험  걸어서 지구 끝까지


380    북극열전-3부 원시북극의 위기(Entering Virgin Territory)
784                             북극열전 - 제4부 원시북극의 위
384              북극열전-6부 영토 분쟁의 서곡(Border Confict)
382             북극열전-4부 남획의 그늘(Predatory activies)
783                              북극열전 - 제3부 남획의 그늘
785                           북극열전 - 제5부 영토 분쟁의 서곡
378                             북극열전-1부 위기의 북극을 가다
379                      북극열전-2부 북극항로, 신해상 실크로드의 꿈
779                                북극열전 - 제1부 북극항로
383                북극열전-5부 신 냉전의 그림자(The War Game)
Name: epi, dtype: object

## Recommender 

In [19]:
def recommender(idx):
    print('episode :'+ str(idx) + ' / ' + metadata['epi'][idx-1])
    
    return get_recommendations(idx, cosine_sim)
    

In [20]:
recommender(1)

episode :1 / 그랜드 슬램 대탐험  걸어서 지구 끝까지


380    북극열전-3부 원시북극의 위기(Entering Virgin Territory)
784                             북극열전 - 제4부 원시북극의 위
384              북극열전-6부 영토 분쟁의 서곡(Border Confict)
382             북극열전-4부 남획의 그늘(Predatory activies)
783                              북극열전 - 제3부 남획의 그늘
785                           북극열전 - 제5부 영토 분쟁의 서곡
378                             북극열전-1부 위기의 북극을 가다
379                      북극열전-2부 북극항로, 신해상 실크로드의 꿈
779                                북극열전 - 제1부 북극항로
383                북극열전-5부 신 냉전의 그림자(The War Game)
Name: epi, dtype: object

In [21]:
recommender(100)

episode :100 / 용서... 그 먼 길 끝에 당신이 있습니까?


826                    무언가족2 - 1부. 벽 너머의 가족
928                 무언가족 3 - 2부 하얀 방, 길을 찾다
734                 무언가족(2부) 하얀집 - 가족의 재탄생 
829                무언가족2 - 2부. 새로운 시작, 하얀 방
1180                       가족쇼크 - 7부 마지막 식사
1339            가족쇼크_ABU 수상 기념_마석, 집으로 가는 길
1174              가족쇼크 - 2부 당신의 가족은 안녕하십니까?
1175                 가족쇼크 - 3부 마석, 집으로 가는 길
414     가정의 달 특집 3부작 가족의 페르소나 - 1부 아버지의 빈 집
1182                 가족쇼크 - 8부 청춘, 고독사를 말하다
Name: epi, dtype: object

# sample data 전체 추천 

In [22]:
for i in metadata['idx']:
    print(recommender(i))
    print('')

episode :1 / 그랜드 슬램 대탐험  걸어서 지구 끝까지
380    북극열전-3부 원시북극의 위기(Entering Virgin Territory)
784                             북극열전 - 제4부 원시북극의 위
384              북극열전-6부 영토 분쟁의 서곡(Border Confict)
382             북극열전-4부 남획의 그늘(Predatory activies)
783                              북극열전 - 제3부 남획의 그늘
785                           북극열전 - 제5부 영토 분쟁의 서곡
378                             북극열전-1부 위기의 북극을 가다
379                      북극열전-2부 북극항로, 신해상 실크로드의 꿈
779                                북극열전 - 제1부 북극항로
383                북극열전-5부 신 냉전의 그림자(The War Game)
Name: epi, dtype: object

episode :2 / 꿈을 쏘다! - 박주영 박지성
233                                         당신은 박지성을 아는가
334                                   추신수, 마침내 메이저리그를 치다
1462                           리우올림픽 특집 박세리, 내 생에 최고의 순간
1243                                      슈틸리케, 리더십을 부탁해
27                                 ‘D-103, 축구 국가대표 전훈기록’
460     <2010 MBC 현대사 연속 기획>경술국치 100년 특집“축구 그리고 세 개의 조국”
1152                                   The Boxer 미치면 이긴다

Name: epi, dtype: object

episode :81 / 알파걸, 그들이 빛나는 이유
1226                           시대의 벽을 넘은 여성
349                            브래지어 하고 계세요?
637                        아랍 여인, 금기 너머를 보다
1347                                발칙한 그녀들
1307           또 하나의 독립운동 - 3부 주걱 대신 총을 들다!
1305    또 하나의 독립운동 - 1부 비밀결사의 여인들, 독립을 외치다!
1306       또 하나의 독립운동 - 2부 나라를 구하는데 귀천은 없다!
11                             깨어나는 이슬람의 딸들
493              2부작 산후조리의 비밀 - 1부 산후풍은 있는가
1498                            불안한 나라의 앨리스
Name: epi, dtype: object

episode :82 / 몰입, 최고의 나를 만난다
31          행복 선진국의 비결 - 가족이 행복한 나라
1260                  우리가 행복할 수 있을까
859          행복의 조건, 복지국가를 가다 1부 노동
135                    자연주의 마을 토트네스
553                   지리산에서 행복을 배우다
908       적게 벌고 더 잘 사는 법 - 도시부족의 탄생
1060        숲으로 간 사람들 2부 - 새 인생을 얻다
1615    행복한 주거 2부 당신의 마지막 집은 어디입니까?
1636         퇴근 후에 뭐하세요? - 사생활의 달인들
861          행복의 조건, 복지국가를 가다 3부 주거
Name: epi, dtype: object

episode :83 / 신영복 교수의 금강산 사색
164          길을 찾는 사람들 - 휴메니버시티 -


Name: epi, dtype: object

episode :128 / 매혹의 에너지,색
156                    당신의 과학 - 제1부 당신은 누구세요?
72                          0.2평의 기적, 절하는 사람들
910                             K-POP의 숨겨진 비밀
1796                       종전 - 한반도 평화의 문을 열다
157                   당신의 과학 - 제2부 당신의 행복한 시간
158                    당신의 과학 - 제3부 당신과 세상 사이
1395                                커뮤니케이션의 힘
1448     한국의 과학과 문명위대한 유산(3)조선 천문학 - 하늘을 개작하라
1531    행복한 국가를 만든 리더십(1)메르켈 리더십 - 국민의 마음을 얻다
625                  과학혁명의 이정표 제5부 생명의 사슬, 유전
Name: epi, dtype: object

episode :129 / 인터뷰다큐 성장통, '만남'
129                인터뷰다큐 성장통, '나이'
130                 인터뷰다큐 성장통, '꿈'
1294        결혼의 진화 - 2부 결혼, 하셨습니까?
1499        ‘우리가 결혼하지 않는 진짜 이유’ 2부
1295        결혼의 진화 - 3부 새로운 가족이 온다
1292          결혼의 진화 - 1부 인류, 결혼하다
1489    부부는 무엇으로 사는가, 3부 일부일처제의 비밀
1495        ‘우리가 결혼하지 않는 진짜 이유’ 1부
1487       부부는 무엇으로 사는가, 1부 부부의 탄생
30          작은 나라의 큰 행복 - 은둔의 땅 부탄
Name: epi, dtype: object

episode :130 / 인터뷰다큐 성장통, '나이'
130             인터뷰다큐 성장통

Name: epi, dtype: object

episode :196 / 아빠 나라를 떠난 아이들
819                            이혼수업
354             2009 가장 슬픈이야기, 풀빵엄마
1439               제 1부. 착한 내 딸의 반란
241       엄마는 무엇으로 사는가 - 고백2. 엄마의 방
440                   나는 한국남자와 결혼했다
1080               아버지의 성 2부 아빠의 역습
1369    아빠 찾는 게 죄인가요? - 코피노의 마지막 선택
1442                제 2부. 엄마처럼 안 살아
240       엄마는 무엇으로 사는가 - 고백1. 엄마의 힘
1351      어떻게 영재가 되는가 섬세한 아빠 터프한 엄마
Name: epi, dtype: object

episode :197 / 하노이, 천년의 붓
78                           2007 라이따이한의 눈물
774                        세상에서 가장 아름다운 일주일
450       조선의 프로페셔널 화인 3부 조선풍속화,세계를 거닐다-김준근
848                         싸이 GO, 강남스타일 신화
155                          대한민국 기적의 열쇠 2부
336                         하늘이 내린 소리 코리안 벨
668    창사 50주년 특별 다큐 “코리아를 아시나요’ - 공공외교의 현장
39             SBS 스페셜 [잃어버린 역사 연개소문](HDTV)
30                   작은 나라의 큰 행복 - 은둔의 땅 부탄
2            메이드 인 코리아 1부 - 광복 60년 한국인들의 위상
Name: epi, dtype: object

episode :198 / 최민수, 죄민수... 그리고 소문
1356                      소문과 거짓말

Name: epi, dtype: object

episode :251 / 피타고라스 정리의 비밀-3부 지구 위의 딱정벌레
249     피타고라스 정리의 비밀-2부 A2+B2=C2
1663        넘버스 - 2부 천국의 사다리, ∞ 
1667          넘버스 - 5부 천공의 수, 허수
1661          넘버스 - 1부 하늘의 수,  π
1664          넘버스 - 3부 자유의 수, 엑스
1666           넘버스 - 4부 신의 손짓, 영
248      피타고라스 정리의 비밀-1부 삼각형의 흔적
1122           문명과 수학 - 3부 신의 숫자
1126         문명과 수학 - 5부 남겨진 문제들
1124    문명과 수학 - 4부 움직이는 세계, 미적분
Name: epi, dtype: object

episode :252 / 동물원 이야기
238            자연으로 돌아간 반달가슴곰 - [모정]
1560                  <사라진 인류> 2부 생존
1740             생존 - 2부 최후의 생존, 너구리
1741              생존 - 3부 새, 생존을 말하다
1739               생존 - 1부 최후의 생존, 삵
753     동물, 행복의 조건 2부 - 험한 세상 다리가 되어
899                        동물원이 살아있다
385                          도시의 유인원
1859                   숲 속의 인간, 오랑우탄
782                           굿모닝 판다
Name: epi, dtype: object

episode :253 / <16인의 성공도전 설득의 비밀>4부 협상으로 진입하라
246            <16인의 성공도전 설득의 비밀>3부 설득 레이더를 맞춰라
245               <16인의 성공도전 설득의 비밀>2부 달인에게 배운다
244        <16인

...

episode :1442 /  젊음도 성형할 수 있나요
49                           얼굴
929       그녀, 뼈를 깎다 - 내 딸의 양악수술
86                           미인
722              내 안에 숨은 6가지 얼굴
1390     성형외과 의사의 고백 - 유령수술의 실체
774            세상에서 가장 아름다운 일주일
1120                     동안의 비밀
1624     황금비율의 비밀 - 1부 숨은 그림 찾기
1255     병원의 고백1부 - 너무나 친절한 의사들
1652              블루존 - 암을 잊은 섬
Name: epi, dtype: object

episode :1443 / 제 2부. 엄마처럼 안 살아
1439                제 1부. 착한 내 딸의 반란
242      엄마는 무엇으로 사는가 - 고백3. 엄마의 짝사랑
241        엄마는 무엇으로 사는가 - 고백2. 엄마의 방
240        엄마는 무엇으로 사는가 - 고백1. 엄마의 힘
715                 마더 쇼크 1부 모성의 대물림
216     모성탐구 엄마가 달라졌어요 제1편 초보엄마 아카데미
716          마더 쇼크 2부 엄마 뇌 속에 아이가 있다
717                  마더 쇼크 3부 나는 엄마다
323                        마흔에 엄마 되다
1662                  도마일기2 - 꽃다운 날들
Name: epi, dtype: object

episode :1444 / 버려지는 아이들 베이비박스60일의 기록
464      이야기의 힘 1부 이야기 오딧세이, History = Story
573                           잠 - 1부 인생의 첫 잠
466                      이야기의 힘 3부 스토리텔링의 시대
465     이야기의 힘 2부 이야기의

Name: epi, dtype: object

episode :1597 / 삶의 기술 - 나이듦에 대하여
1771    100세 쇼크 2부 장수의 역습 벼랑 끝에 선 노인들
1188           나는 산다. 이선희, 대한민국을 위로하다
1464                       몰카 천국 대한민국
1279                     나는 대한민국 여군이다
381                                치킨
1772      100세 쇼크 3부 늙은 신인류의 진짜 노인 되기
1770        100세 쇼크 1부 관찰기록 100세의 사생활
1569         글로벌 인재전쟁 5부 대한민국, 도약의 조건
1417                 노후 파산 - 당신의 노후는?
532                2011 신년특집 안철수와 박경철
Name: epi, dtype: object

episode :1598 / 섬진강 초딩들의 대선일기
1561                          사건번호 2016헌나1
1542                              대통령의 탄생 
1591        <대통령은 누구인가> 2부. 위 더 피플, 국민의 권리
883                    학교폭력 - 제6부 어른들이 미안해
354                    2009 가장 슬픈이야기, 풀빵엄마
1589    <대통령은 누구인가> 1부. 미스터 프레지던트, 대통령의 탄생
374                       아이의 밥상 1부 편식의 비밀
819                                   이혼수업
1595                      2017 대한민국 민심 보고서
822                        킹메이커 1부 네거티브 전쟁
Name: epi, dtype: object

episode :1599 / 무병장수 - 골든 타임
1338      밥상! 상식을 뒤집

Name: epi, dtype: object

episode :1654 / <철학하라> 제1부 - 너희는 왜 우리를 미워하는가
1655             <철학하라> 제3부 - 왜 그대가 옳은가?
1660            <철학하라> 제5부 - 철학의 시원, 그리스
1659                 <철학하라> 제4부 - 이데아의 꿈
1654                 <철학하라> 제2부 - 마음의 조건
1317        우리 WE, 3부 우리는 왜 우리가 되지 못하는가?
1407                           하늘에서 본 암자
30                작은 나라의 큰 행복 - 은둔의 땅 부탄
1117    미래를 바꾸는 교육 - 3부 경제, 자본주의를 살아가는 힘
316      예술, 일상을 만나다 - 2부 아주 사적인, 아주 공적인
1811                <맛의 배신> 2부-중독을 부르는 향
Name: epi, dtype: object

episode :1655 / <철학하라> 제2부 - 마음의 조건
1655                  <철학하라> 제3부 - 왜 그대가 옳은가?
1660                 <철학하라> 제5부 - 철학의 시원, 그리스
1653             <철학하라> 제1부 - 너희는 왜 우리를 미워하는가
1659                      <철학하라> 제4부 - 이데아의 꿈
1745                     <4차 인간> 2부 인간은 기계인가?
1744                <4차 인간> 1부 우리는 영원할 수 있을까?
1746              <4차 인간> 3부 어떻게 기계와 공존할 것인가?
1649    <4차 산업혁명시대 교육대혁명> 1부 AI와 인간의 공존은 가능한가
30                     작은 나라의 큰 행복 - 은둔의 땅 부탄
1117         미래를 바꾸는 교육 - 3부 경제, 자본주의를 살아가는

Name: epi, dtype: object

episode :1768 / 누가 북한을 움직이는가(1)평양의 파워 엘리트
948                        최초공개 북한의 백두대간
1790          남북정상회담 특집 다큐 <테이블 너머의 김정은>
1880     방송의 날 특집6개월 후 만납시다: 북한 결핵병원 이야기
1809                    84년생 김정은과 장마당 세대
1640                           김정은의 두 얼굴
271                            연평별곡 (別曲)
165                              말도 아리랑 
98      남측 벼를 심으려 왔습니다 - 평양, 당곡리에서 맺은 결실
202                 5년의 기록 - 압록강 二千里 사람들
1819          담 판 - 트럼프 vs 김정은, 그들의 선택은?
Name: epi, dtype: object

episode :1769 / 권력과 거짓말 (부제: 피노키오의 나라)
1548                       시크릿 공화국
1588                        권력의 탄생
1626    황금비율의 비밀- 2부 절대적이고 상대적인 진리
978          죽음의 습격자 - 후쿠시마발 방사능공포
721                   -화무십일홍 권불십년-
438                      김장훈의 개똥철학
159            SBS스페셜 다큐멘터리 동화 네팔 
808            자본주의 3부. 금융지능은 있는가?
368                  영  동  선 (嶺東線)
367         출세 만세 - 4부 리더에게 길을 묻다.
Name: epi, dtype: object

episode :1770 / <10년 후의 세계> 2부 잉여 인간, 저항의 시작
1762    <10년 후의 세계> 1부 멋진 신세계와 일자리 도

Name: epi, dtype: object

episode :1827 / 6월항쟁 특집 : 어머니와 사진사
137                                  서울은 사랑할 것이 많다
417     가정의 달 특집 3부작 가족의 페르소나 - 3부 자식, 가시방석 위의 캥거루
24                       한국을 떠난 사람들 - 그들이 말하는 대한민국
1172                           가족쇼크 - 1부 나는 부모입니다.
440                                  나는 한국남자와 결혼했다
1339                   가족쇼크_ABU 수상 기념_마석, 집으로 가는 길
167                                  기억 속으로 떠나는 여행
578             1935코레아-스텐베리만의 기억 2부 잃어버린 땅 이어지는 삶
1643                        오늘을 잊지 마 - 사라진 25살의 기억
1517                            세월의 가치를 더하다 - 세운상가
Name: epi, dtype: object

episode :1828 / <한국사 오천년, 생존의 길> 제1부 - 신라는 어떻게 살아남았나?
1726    <한국사 오천년, 생존의 길> 제4부 - 병자호란, 궁지에 빠진 중립외교
1723        <한국사 오천년, 생존의 길> 제3부 - 몽골제국과의 이상한 전쟁
1829          <한국사 오천년, 생존의 길> 제6부 - 패권교체기의 생존전략
1828    <한국사 오천년, 생존의 길> 제2부 - 거란전쟁, 동북아 균형자의 조건
1727       <한국사 오천년, 생존의 길> 제5부 - 고종, 열강의 덫에 빠지다
1739                           생존 - 1부 최후의 생존, 삵
1740                         생존 - 2부 최후의 생존, 너구리
1560  

# =====추천시스템 결과=====

In [156]:
def get_recommendations(idx, cosine_sim=cosine_sim):
    #제목과 일치하는 에피소드 인덱스 가지고옴
    idx = indices[idx]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #비슷한 에피소드 상위 5개
    sim_scores = sim_scores[1:11]
    
    # indices
    epi_indices = [i[0] for i in sim_scores]
    print(epi_indices)
    get_result(idx,epi_indices)
    return metadata['epi'].iloc[epi_indices]

In [157]:
result_data = []
def get_result(idx, epi_indices):
    recom_set = epi_indices.copy()
    recom_set = [ x+1 for x in recom_set ]
    recom_set.insert(0, idx+1)
    print(recom_set)
    result_data.append(recom_set)

In [158]:

size = metadata.shape[0]
for i in range(1,size+1):
    get_recommendations(i)
    print('\n')

[380, 784, 384, 382, 783, 785, 378, 379, 779, 383]
[1, 381, 785, 385, 383, 784, 786, 379, 380, 780, 384]


[233, 334, 1462, 1243, 27, 460, 1152, 391, 1880, 1309]
[2, 234, 335, 1463, 1244, 28, 461, 1153, 392, 1881, 1310]


[7, 155, 848, 336, 151, 6, 293, 29, 54, 762]
[3, 8, 156, 849, 337, 152, 7, 294, 30, 55, 763]


[7, 155, 5, 2, 23, 293, 363, 691, 266, 475]
[4, 8, 156, 6, 3, 24, 294, 364, 692, 267, 476]


[527, 1163, 1160, 1162, 366, 54, 360, 367, 531, 155]
[5, 528, 1164, 1161, 1163, 367, 55, 361, 368, 532, 156]


[1057, 848, 1054, 29, 1056, 7, 538, 2, 952, 1435]
[6, 1058, 849, 1055, 30, 1057, 8, 539, 3, 953, 1436]


[155, 841, 2, 54, 151, 842, 326, 691, 24, 668]
[7, 156, 842, 3, 55, 152, 843, 327, 692, 25, 669]


[2, 24, 500, 499, 1785, 155, 54, 1683, 336, 35]
[8, 3, 25, 501, 500, 1786, 156, 55, 1684, 337, 36]


[1846, 83, 595, 557, 1850, 1361, 1733, 1737, 727, 1478]
[9, 1847, 84, 596, 558, 1851, 1362, 1734, 1738, 728, 1479]


[75, 738, 765, 502, 744, 836, 840, 832, 501, 837]
[10, 76

[31, 1260, 859, 135, 553, 908, 1060, 1615, 1636, 861]
[82, 32, 1261, 860, 136, 554, 909, 1061, 1616, 1637, 862]


[164, 159, 368, 533, 438, 187, 1040, 750, 1138, 532]
[83, 165, 160, 369, 534, 439, 188, 1041, 751, 1139, 533]


[1781, 1543, 595, 294, 468, 684, 1856, 682, 647, 954]
[84, 1782, 1544, 596, 295, 469, 685, 1857, 683, 648, 955]


[1634, 1519, 701, 1524, 735, 1520, 1525, 164, 544, 1745]
[85, 1635, 1520, 702, 1525, 736, 1521, 1526, 165, 545, 1746]


[474, 452, 254, 166, 301, 1852, 403, 1869, 1758, 1777]
[86, 475, 453, 255, 167, 302, 1853, 404, 1870, 1759, 1778]


[49, 1441, 929, 722, 567, 187, 366, 201, 30, 686]
[87, 50, 1442, 930, 723, 568, 188, 367, 202, 31, 687]


[837, 831, 501, 738, 830, 836, 502, 744, 775, 777]
[88, 838, 832, 502, 739, 831, 837, 503, 745, 776, 778]


[463, 461, 162, 45, 709, 957, 449, 1366, 1049, 330]
[89, 464, 462, 163, 46, 710, 958, 450, 1367, 1050, 331]


[368, 940, 30, 1475, 1358, 266, 532, 23, 1855, 951]
[90, 369, 941, 31, 1476, 1359, 267, 533, 24, 185

[520, 782, 346, 251, 106, 1442, 451, 348, 1439, 1859]
[239, 521, 783, 347, 252, 107, 1443, 452, 349, 1440, 1860]


[276, 1034, 1341, 1683, 54, 24, 1309, 1785, 1049, 279]
[240, 277, 1035, 1342, 1684, 55, 25, 1310, 1786, 1050, 280]


[241, 242, 1439, 1442, 216, 716, 715, 717, 323, 593]
[241, 242, 243, 1440, 1443, 217, 717, 716, 718, 324, 594]


[240, 242, 1439, 1442, 216, 716, 715, 717, 323, 354]
[242, 241, 243, 1440, 1443, 217, 717, 716, 718, 324, 355]


[240, 241, 1442, 1439, 715, 216, 717, 716, 217, 323]
[243, 241, 242, 1443, 1440, 716, 217, 718, 717, 218, 324]


[73, 827, 1444, 775, 1635, 159, 519, 1175, 741, 734]
[244, 74, 828, 1445, 776, 1636, 160, 520, 1176, 742, 735]


[245, 246, 253, 252, 1314, 1316, 1319, 893, 1317, 1322]
[245, 246, 247, 254, 253, 1315, 1317, 1320, 894, 1318, 1323]


[244, 246, 253, 252, 1314, 1316, 1319, 1317, 1322, 893]
[246, 245, 247, 254, 253, 1315, 1317, 1320, 1318, 1323, 894]


[244, 245, 253, 252, 1314, 1316, 1319, 1317, 164, 159]
[247, 245, 246, 254, 25

[332, 333, 159, 68, 368, 589, 69, 1855, 1493, 894]
[332, 333, 334, 160, 69, 369, 590, 70, 1856, 1494, 895]


[333, 331, 789, 147, 1785, 29, 1565, 1061, 146, 69]
[333, 334, 332, 790, 148, 1786, 30, 1566, 1062, 147, 70]


[332, 331, 868, 873, 867, 869, 872, 871, 1878, 113]
[334, 333, 332, 869, 874, 868, 870, 873, 872, 1879, 114]


[64, 1384, 1386, 1385, 414, 647, 1339, 1289, 1172, 977]
[335, 65, 1385, 1387, 1386, 415, 648, 1340, 1290, 1173, 978]


[339, 342, 156, 25, 1034, 157, 650, 1252, 158, 960]
[336, 340, 343, 157, 26, 1035, 158, 651, 1253, 159, 961]


[789, 2, 279, 668, 7, 848, 794, 147, 468, 1448]
[337, 790, 3, 280, 669, 8, 849, 795, 148, 469, 1449]


[1781, 684, 354, 545, 682, 185, 374, 539, 352, 186]
[338, 1782, 685, 355, 546, 683, 186, 375, 540, 353, 187]


[1010, 124, 962, 620, 1397, 324, 385, 479, 1571, 1838]
[339, 1011, 125, 963, 621, 1398, 325, 386, 480, 1572, 1839]


[335, 1034, 342, 650, 279, 30, 48, 1811, 307, 1682]
[340, 336, 1035, 343, 651, 280, 31, 49, 1812, 308, 1683]

[445, 445, 446, 605, 606, 1222, 604, 1120, 609, 1060, 119]


[443, 444, 604, 605, 1221, 603, 1119, 608, 1059, 118]
[446, 444, 445, 605, 606, 1222, 604, 1120, 609, 1060, 119]


[441, 408, 1709, 464, 466, 684, 465, 744, 738, 1000]
[447, 442, 409, 1710, 465, 467, 685, 466, 745, 739, 1001]


[807, 922, 924, 30, 1477, 31, 908, 1117, 1260, 135]
[448, 808, 923, 925, 31, 1478, 32, 909, 1118, 1261, 136]


[450, 449, 96, 330, 570, 569, 88, 568, 163, 162]
[449, 451, 450, 97, 331, 571, 570, 89, 569, 164, 163]


[448, 450, 88, 568, 570, 569, 232, 1315, 330, 1490]
[450, 449, 451, 89, 569, 571, 570, 233, 1316, 331, 1491]


[448, 449, 88, 96, 330, 612, 1053, 30, 34, 570]
[451, 449, 450, 89, 97, 331, 613, 1054, 31, 35, 571]


[348, 346, 1739, 1560, 1740, 1741, 547, 1263, 546, 1262]
[452, 349, 347, 1740, 1561, 1741, 1742, 548, 1264, 547, 1263]


[459, 454, 474, 85, 166, 254, 301, 1852, 403, 1869]
[453, 460, 455, 475, 86, 167, 255, 302, 1853, 404, 1870]


[1559, 594, 458, 1550, 1557, 1510, 1552, 1514, 22

...

[1250, 1251, 1190, 1248, 1189, 710, 708, 1677, 1136, 1183]
[1192, 1251, 1252, 1191, 1249, 1190, 711, 709, 1678, 1137, 1184]


[464, 1730, 1731, 1732, 826, 829, 1180, 466, 465, 416]
[1193, 465, 1731, 1732, 1733, 827, 830, 1181, 467, 466, 417]


[848, 499, 500, 668, 789, 794, 183, 425, 1195, 1100]
[1194, 849, 500, 501, 669, 790, 795, 184, 426, 1196, 1101]


[803, 277, 809, 528, 529, 718, 278, 1017, 295, 1016]
[1195, 804, 278, 810, 529, 530, 719, 279, 1018, 296, 1017]


[1279, 155, 151, 1193, 1188, 1464, 1569, 54, 499, 500]
[1196, 1280, 156, 152, 1194, 1189, 1465, 1570, 55, 500, 501]


[1110, 1157, 1206, 1155, 1212, 1424, 1586, 1444, 1303, 355]
[1197, 1111, 1158, 1207, 1156, 1213, 1425, 1587, 1445, 1304, 356]


[1294, 1292, 1495, 1499, 1295, 1487, 740, 599, 1630, 1489]
[1198, 1295, 1293, 1496, 1500, 1296, 1488, 741, 600, 1631, 1490]


[1003, 1179, 756, 1325, 732, 1078, 1077, 862, 269, 999]
[1199, 1004, 1180, 757, 1326, 733, 1079, 1078, 863, 270, 1000]


[1200, 1205, 1210, 1203, 1211, 1202

[1281, 1286, 1279, 1282, 1284, 1287, 242, 1440, 1443, 243, 241]


[1285, 1278, 1280, 1283, 1286, 1643, 47, 167, 1169, 1309]
[1282, 1286, 1279, 1281, 1284, 1287, 1644, 48, 168, 1170, 1310]


[1393, 1814, 1325, 819, 1625, 1630, 92, 144, 563, 665]
[1283, 1394, 1815, 1326, 820, 1626, 1631, 93, 145, 564, 666]


[1281, 1285, 1278, 1280, 1286, 22, 47, 339, 1311, 1317]
[1284, 1282, 1286, 1279, 1281, 1287, 23, 48, 340, 1312, 1318]


[848, 1381, 34, 1100, 1193, 763, 5, 564, 789, 2]
[1285, 849, 1382, 35, 1101, 1194, 764, 6, 565, 790, 3]


[1278, 1280, 1281, 1283, 1286, 1317, 1680, 339, 750, 1308]
[1286, 1279, 1281, 1282, 1284, 1287, 1318, 1681, 340, 751, 1309]


[1278, 1280, 1285, 1281, 1283, 1746, 1745, 1308, 981, 1744]
[1287, 1279, 1281, 1286, 1282, 1284, 1747, 1746, 1309, 982, 1745]


[1515, 211, 945, 474, 166, 461, 45, 1638, 403, 279]
[1288, 1516, 212, 946, 475, 167, 462, 46, 1639, 404, 280]


[1036, 1605, 111, 1607, 187, 1659, 1047, 228, 229, 884]
[1289, 1037, 1606, 112, 1608, 188, 1660, 104


[1128, 1131, 1451, 279, 45, 458, 557, 923, 461, 957]
[1448, 1129, 1132, 1452, 280, 46, 459, 558, 924, 462, 958]


[336, 279, 311, 45, 957, 612, 605, 463, 1061, 312]
[1449, 337, 280, 312, 46, 958, 613, 606, 464, 1062, 313]


[111, 1446, 641, 88, 449, 1474, 1475, 330, 228, 1621]
[1450, 112, 1447, 642, 89, 450, 1475, 1476, 331, 229, 1622]


[1641, 1255, 1390, 1258, 940, 1808, 1013, 1296, 1670, 1636]
[1451, 1642, 1256, 1391, 1259, 941, 1809, 1014, 1297, 1671, 1637]


[1447, 1771, 1772, 279, 45, 1535, 1770, 1128, 1131, 92]
[1452, 1448, 1772, 1773, 280, 46, 1536, 1771, 1129, 1132, 93]


[545, 813, 706, 1709, 832, 9, 906, 1376, 765, 350]
[1453, 546, 814, 707, 1710, 833, 10, 907, 1377, 766, 351]


[27, 1152, 1454, 1773, 1462, 788, 820, 1331, 177, 701]
[1454, 28, 1153, 1455, 1774, 1463, 789, 821, 1332, 178, 702]


[770, 788, 1444, 1773, 623, 1462, 1453, 1728, 438, 1152]
[1455, 771, 789, 1445, 1774, 624, 1463, 1454, 1729, 439, 1153]


[1873, 1728, 1236, 1878, 1454, 1773, 770, 1161, 62, 1861]
[1

[1614, 1310, 1622, 1034, 235, 1035, 1683, 1692, 290, 1050, 343]


[1615, 1616, 1392, 861, 94, 1235, 1197, 128, 464, 1354]
[1615, 1616, 1617, 1393, 862, 95, 1236, 1198, 129, 465, 1355]


[1616, 1614, 31, 861, 24, 859, 135, 553, 908, 1260]
[1616, 1617, 1615, 32, 862, 25, 860, 136, 554, 909, 1261]


[1615, 1614, 1233, 1234, 1231, 40, 861, 795, 30, 128]
[1617, 1616, 1615, 1234, 1235, 1232, 41, 862, 796, 31, 129]


[377, 1810, 467, 1160, 1811, 374, 1708, 422, 1162, 496]
[1618, 378, 1811, 468, 1161, 1812, 375, 1709, 423, 1163, 497]


[1581, 761, 1807, 164, 438, 1513, 91, 1036, 26, 160]
[1619, 1582, 762, 1808, 165, 439, 1514, 92, 1037, 27, 161]


[279, 1407, 48, 1608, 30, 636, 1317, 1034, 1654, 1451]
[1620, 280, 1408, 49, 1609, 31, 637, 1318, 1035, 1655, 1452]


[1373, 855, 1825, 1581, 991, 1476, 265, 1589, 1722, 1401]
[1621, 1374, 856, 1826, 1582, 992, 1477, 266, 1590, 1723, 1402]


[1309, 1613, 1049, 342, 1034, 234, 289, 1691, 1033, 473]
[1622, 1310, 1614, 1050, 343, 1035, 235, 290, 1692, 1


[1775, 1777, 1776, 1782, 1839, 1840, 1064, 474, 546, 1262]
[1781, 1776, 1778, 1777, 1783, 1840, 1841, 1065, 475, 547, 1263]


[83, 1543, 684, 337, 595, 468, 682, 294, 1856, 1172]
[1782, 84, 1544, 685, 338, 596, 469, 683, 295, 1857, 1173]


[1775, 1777, 1780, 1776, 1839, 1840, 1064, 474, 546, 1262]
[1783, 1776, 1778, 1781, 1777, 1840, 1841, 1065, 475, 547, 1263]


[286, 622, 69, 1671, 164, 159, 100, 1855, 266, 66]
[1784, 287, 623, 70, 1672, 165, 160, 101, 1856, 267, 67]


[1754, 1552, 458, 1559, 1447, 187, 477, 1884, 1468, 481]
[1785, 1755, 1553, 459, 1560, 1448, 188, 478, 1885, 1469, 482]


[1435, 147, 1565, 789, 1061, 35, 660, 29, 146, 662]
[1786, 1436, 148, 1566, 790, 1062, 36, 661, 30, 147, 663]


[1789, 1788, 315, 316, 1654, 1745, 30, 1744, 1746, 154]
[1787, 1790, 1789, 316, 317, 1655, 1746, 31, 1745, 1747, 155]


[1791, 1815, 1696, 1111, 350, 1339, 1172, 801, 1525, 1397]
[1788, 1792, 1816, 1697, 1112, 351, 1340, 1173, 802, 1526, 1398]


[1786, 1789, 315, 316, 1654, 442, 185, 1745

In [152]:
index_name = ['index']
for i in range(10):
    index_name.append('reco'+str(i+1))
index_name

['index',
 'reco1',
 'reco2',
 'reco3',
 'reco4',
 'reco5',
 'reco6',
 'reco7',
 'reco8',
 'reco9',
 'reco10']

In [159]:
df_result = pd.DataFrame(result_data, columns = index_name)
df_result

,index,reco1,reco2,reco3,reco4,reco5,reco6,reco7,reco8,reco9,reco10
0,1,381,785,385,383,784,786,379,380,780,384
1,2,234,335,1463,1244,28,461,1153,392,1881,1310
2,3,8,156,849,337,152,7,294,30,55,763
3,4,8,156,6,3,24,294,364,692,267,476
4,5,528,1164,1161,1163,367,55,361,368,532,156
5,6,1058,849,1055,30,1057,8,539,3,953,1436
6,7,156,842,3,55,152,843,327,692,25,669
7,8,3,25,501,500,1786,156,55,1684,337,36
8,9,1847,84,596,558,1851,1362,1734,1738,728,1479
9,10,76,739,766,503,745,837,841,833,502,838


In [165]:
df_result.to_csv('recommender_system_web.csv', encoding='utf-8', index=None)